In [66]:
pip install pyLDAvis


     ---------------------------------------- 2.6/2.6 MB 2.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 298.0/298.0 kB 9.3 MB/s eta 0:00:00
     ---------------------------------------- 92.7/92.7 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0


In [1]:
import jieba
import os
import re
import csv
import pyLDAvis.gensim_models
from gensim import corpora,models,similarities

In [2]:
#创建停用词列表
def stopwordslist():
    stopwords = [line.strip() for line in open('C:/Users/11040/Desktop/weibonlp-master/data/stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords

#对句子进行中文分词
def seg_depart(sentence):
    sentence_depart = jieba.cut(sentence.strip())
    stopwords = stopwordslist()
    outstr = ''
    for word in sentence_depart:
        if word not in stopwords:
            outstr += word
            outstr += " "   
    return outstr

csv_reader = csv.reader(open("C:/Users/11040/Desktop/weibonlp-master/data/combined_csv.csv"))
outfilename = "C:/Users/11040/Desktop/weibonlp-master/data/0219/train_jieba.txt"
outputs = open(outfilename, 'w', encoding='UTF-8')

# 去除非汉字的字符
for line in csv_reader:
    line = line[3]
    line = re.sub(r'[^\u4e00-\u9fa5]+','',line)
    line_seg = seg_depart(line.strip())
    outputs.write(line_seg.strip() + '\n')
    
outputs.close()

print("删除停用词和分词完毕")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\11040\AppData\Local\Temp\jieba.cache
Loading model cost 0.720 seconds.
Prefix dict has been built successfully.


删除停用词和分词完毕


In [34]:
f = open('C:/Users/11040/Desktop/weibonlp-master/data/0219/train_jieba.txt', 'r',encoding='utf-8')
train = []
for line in f.readlines():
    line = [word.strip() for word in line.split(' ')]
    train.append(line)

print(train)


[['五十岁', '列', '真的', '感动', '真的', '相信', '真有', '时刻', '事情', '发生'], ['电影院', '哭', '成', '傻狗'], ['中国', '骨子里', '真正', '人类', '大同', '信仰', '关键时刻', '义无反顾', '选择', '自我牺牲', '无悔', '成就', '大局'], ['面壁', '刘德华', '破壁'], ['双腿', '机器人', '巨浪', '来袭', '瞬间', '冲到', '周', '老师', '身前', '开启', '防御', '模式', '帅', '一批', '开学', '季'], ['发现', '有意思', '一点', '流浪', '地球', '打星', '阿凡达', '打星'], ['普通人', '故事', '续写', '宏大', '叙事', '讲述', '文明', '传承', '史诗'], ['周先生', '那条', '线', '带入', '总理', '一幕', '特别', '乔', '笑', '爱'], ['周先生', '点火', '那段', '完', '彩蛋', '回去', '看法', '周先生', '这是', '预言', '反复', '摄像头', '角度', '看着', '周先生', '数字', '生命', '人类', '预言', '简单'], ['马兆', '角色', '没提', '其他人', '说', '私心', '支持', '拼死', '工作', '宏大', '信念', '文明', '毫无意义', '死', '完整', '死去', '角色', '值得', '分析'], ['恕我直言', '西方', '鼎盛时期', '作品', '中', '全人类', '共同体', '团结', '面对', '挑战', '设定', '集', '全人类', '之力', '美西', '领导', '统治', '上帝', '之名', '形事', '感觉', '小破球', '想', '传达', '精神', '不太', '真正', '人类', '命运', '共同体', '理念', '千年', '大一统', '思想', '中国', '人才', '更好', '表达'], ['沙发'], ['几个', '小时', '更新', '生产队', '牛', '歇', '流浪', '地球', '赶紧',

In [71]:
import pandas as pd
"""构建词频矩阵，训练LDA模型"""
dictionary = corpora.Dictionary(train)
# 基于词典，使【词】→【稀疏向量】，并将向量放入列表，形成【稀疏向量集】

dictionary.filter_n_most_frequent(4)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000) 

corpus = [dictionary.doc2bow(text) for text in train]

lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5)
topic_list = lda.print_topics(5)

print("5个主题的单词分布为：\n")

for topic in topic_list:
    print(topic)


5个主题的单词分布为：

(0, '0.058*"电影" + 0.030*"中国" + 0.021*"导演" + 0.021*"拍" + 0.020*"郭导" + 0.019*"想" + 0.013*"太" + 0.013*"希望" + 0.012*"分" + 0.012*"小破球"')
(1, '0.035*"说" + 0.027*"允悲" + 0.018*"懂" + 0.016*"完" + 0.016*"第一部" + 0.013*"小时" + 0.012*"二刷" + 0.012*"电影" + 0.012*"拍" + 0.012*"太"')
(2, '0.065*"郭导" + 0.032*"小破球" + 0.021*"休息" + 0.017*"好好" + 0.016*"加油" + 0.015*"太" + 0.014*"喜欢" + 0.013*"确实" + 0.013*"不错" + 0.012*"辛苦"')
(3, '0.031*"苦涩" + 0.022*"睡着" + 0.019*"" + 0.019*"泪" + 0.018*"导演" + 0.017*"中国" + 0.017*"电影" + 0.017*"科幻" + 0.017*"值得" + 0.017*"说"')
(4, '0.026*"完" + 0.026*"哈哈哈" + 0.020*"中国" + 0.019*"泪泪" + 0.018*"数字" + 0.018*"想" + 0.018*"第三部" + 0.016*"人类" + 0.016*"期待" + 0.016*"苦涩"')


In [72]:
plot =pyLDAvis.gensim_models.prepare(lda,corpus,dictionary)
# 保存到本地html
pyLDAvis.save_html(plot, 'C:/Users/11040/Desktop/weibonlp-master/data/pyLDAvis.html')
#探索主题间距图（Intertopic Distance Plot ）可以帮助你了解主题之间的关系，包括主题组之间潜在的高级结构。

C:\Users\11040\AppData\Local\Programs\Python\Python310\Lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [76]:
pyLDAvis.display(plot)